In [1]:
import pandas as pd
import numpy as np

In [2]:
fraud_oracle = pd.read_csv('fraud_oracle.csv')

In [3]:
fraud_oracle = fraud_oracle[ ~ ((fraud_oracle.MonthClaimed == '0') | (fraud_oracle.DayOfWeekClaimed == '0'))]

In [4]:
fraud_oracle

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,...,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,...,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,...,7 years,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision
3,Jun,2,Saturday,Toyota,Rural,Friday,Jul,1,Male,Married,...,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability
4,Jan,5,Monday,Honda,Urban,Tuesday,Feb,2,Female,Single,...,5 years,31 to 35,No,No,External,none,no change,1 vehicle,1994,Collision
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15415,Nov,4,Friday,Toyota,Urban,Tuesday,Nov,5,Male,Married,...,6 years,31 to 35,No,No,External,none,no change,1 vehicle,1996,Collision
15416,Nov,5,Thursday,Pontiac,Urban,Friday,Dec,1,Male,Married,...,6 years,31 to 35,No,No,External,more than 5,no change,3 to 4,1996,Liability
15417,Nov,5,Thursday,Toyota,Rural,Friday,Dec,1,Male,Single,...,5 years,26 to 30,No,No,External,1 to 2,no change,1 vehicle,1996,Collision
15418,Dec,1,Monday,Toyota,Urban,Thursday,Dec,2,Female,Married,...,2 years,31 to 35,No,No,External,more than 5,no change,1 vehicle,1996,All Perils


In [5]:
def get_translator(data):
    translator = {}
    for col in data.columns:
        old_values = data[col].unique().tolist()
        new_values = [1 + i for i in range(len(data[col].unique()))]
        if (data[col].dtypes == object):
            if ('0' in old_values):
                new_values[old_values.index('0')] = np.nan
            # data_profileprint(old_values, new_values)
            translator[col] = [old_values, new_values]
        else:
            translator[col] = [old_values, old_values[:]]
    if ('Age' in data.columns):
        translator['Age'][1][translator['Age'][0].index(0)] = np.nan
    return translator

In [6]:
translator = get_translator(fraud_oracle)

In [7]:
def change_values(data, translator, flag=0):
    for col in data.columns:
        data[col] = data[col].replace(translator[col][flag + 0],
                                      translator[col][1 - flag])
    return data

In [8]:
fraud_oracle = change_values(fraud_oracle, translator, 0)

In [9]:
# from pandas_profiling import ProfileReport

# data_profile = ProfileReport(df=data, title="Analysis of the Data dataframe", explorative=True)
# data_profile

In [10]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=10, weights="uniform")
after_knn = imputer.fit_transform(fraud_oracle)

In [11]:
for i in fraud_oracle[fraud_oracle.isna().Age].index:
    fraud_oracle['Age'][i] = round(after_knn[i][10])

<ipython-input-11-55f53736d1d3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fraud_oracle['Age'][i] = round(after_knn[i][10])


In [12]:
fraud_oracle = change_values(fraud_oracle, translator, 1)

In [22]:
for col in fraud_oracle.columns:
    print(col)



Month
WeekOfMonth
DayOfWeek
Make
AccidentArea
DayOfWeekClaimed
MonthClaimed
WeekOfMonthClaimed
Sex
MaritalStatus
Age
Fault
VehicleCategory
VehiclePrice
FraudFound_P
RepNumber
Deductible
DriverRating
Days_Policy_Accident
Days_Policy_Claim
PastNumberOfClaims
AgeOfVehicle
AgeOfPolicyHolder
PoliceReportFiled
WitnessPresent
AgentType
NumberOfSuppliments
AddressChange_Claim
NumberOfCars
Year
BasePolicy


KeyError: 'PolicyNumber'

In [21]:
fraud_oracle.drop('PolicyType',axis = 1, inplace = True)

In [24]:
fraud_oracle['BasePolicy']

0         Liability
1         Collision
2         Collision
3         Liability
4         Collision
            ...    
15415     Collision
15416     Liability
15417     Collision
15418    All Perils
15419     Collision
Name: BasePolicy, Length: 15419, dtype: object

In [23]:
fraud_oracle.to_csv("fraud_oracle_preprocessed_drop.csv")